In [ ]:

ds = xr.open_dataset(DTU21_path)
mss_lon = ds['lon'].values
mss_lat = ds['lat'].values
mss_data = ds['mss'].values


In [ ]:

linear_interpolator = RegularGridInterpolator((mss_lat, mss_lon), mss_data, method='linear', bounds_error=False, fill_value=np.nan)
nearest_interpolator = RegularGridInterpolator((mss_lat, mss_lon), mss_data, method='nearest', bounds_error=False, fill_value=np.nan)

# 用 MSS 在points上进行线性插值，结果为nan的值再用最邻近方法填补

def combined_interpolation(icesat2_df):
    
    points = np.array(list(zip(icesat2_df['Latitude'], icesat2_df['Longitude'])))
    mss_values_linear = linear_interpolator(points)
    mask_nan = np.isnan(mss_values_linear)
    mss_values_nearest = nearest_interpolator(points[mask_nan])
    mss_values_linear[mask_nan] = mss_values_nearest

    return mss_values_linear


In [ ]:

folder_path = ATL12_path
def read_ICESat2_h5(folder_path):

    beam_map = {
        'beam1': {'forward': 'gt3r', 'backward': 'gt1l'},
        'beam3': {'forward': 'gt2r', 'backward': 'gt2l'},
        'beam5': {'forward': 'gt1r', 'backward': 'gt3l'}
    }

    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.h5')]
    
    for file_path in file_paths:
        try:
            file_name = os.path.basename(file_path).split('_')[1]
            date_str = file_name[:14]
            date_obj = datetime.strptime(date_str, '%Y%m%d%H%M%S')

            with h5py.File(file_path, 'r') as h5_file:
                sc_orient = h5_file['orbit_info/sc_orient'][:]
                if 2 in sc_orient: #transition
                    continue

                cycle_number = h5_file['orbit_info/cycle_number'][0]
                sc_orient_mode = sc_orient[0]
                rgt_numbers = h5_file['orbit_info/rgt'][:]
                rgt_str = '-'.join(map(str, rgt_numbers))
                for beam, mapping in beam_map.items():
                    beam_name = mapping['forward'] if sc_orient_mode == 1 else mapping['backward']
                    
                    lon = h5_file[f'{beam_name}/ssh_segments/longitude'][:]
                    lat = h5_file[f'{beam_name}/ssh_segments/latitude'][:]
                    h = h5_file[f'{beam_name}/ssh_segments/heights/h'][:]
                    tide_ocean_seg = h5_file[f'{beam_name}/ssh_segments/stats/tide_ocean_seg'][:]
                    tide_load_seg = h5_file[f'{beam_name}/ssh_segments/stats/tide_load_seg'][:]
                    dac_seg = h5_file[f'{beam_name}/ssh_segments/stats/dac_seg'][:]

                    data = {
                        'Longitude': lon,
                        'Latitude': lat,
                        'SSH': h,
                        'Tide_Ocean': tide_ocean_seg,
                        'Tide_Load': tide_load_seg,
                        'DAC': dac_seg,
                        'Date': [date_obj] * len(lon)
                    }
                    df = pd.DataFrame(data)

             
                    df = df[(df['Longitude'] >= 100) & (df['Longitude'] <= 140) &
                            (df['Latitude'] >= 0) & (df['Latitude'] <= 50)]
                    
                    if not df.empty:
                        # 计算SSHA & TWLE
                        mss_values = combined_interpolation(df)
                        df['MSS'] = combined_interpolation(df)
                        df['SSHA_DTU21'] = df['SSH'] - mss_values
                        df['TWLE'] = df['SSH'] - df['MSS']+ df['Tide_Ocean'] + df['DAC'] + df['Tide_Load']
                   
                        cycle_folder = os.path.join(folder_path, f"cycle_{cycle_number}", beam)
                        os.makedirs(cycle_folder, exist_ok=True)
                        
        
                        output_file = os.path.join(cycle_folder, f"{beam}_cycle_{cycle_number}_{date_str}_{rgt_str}.csv")
                        df.to_csv(output_file, index=False)
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
            